In [72]:
import re
import random

In [2]:
def contains(a,b):
    a = list(a)
    b = list(b)
    result = True
    for element in a:
        result = result and (element in b)
    return result

In [23]:
def replace_negated(sentence):
    #remove negations from sentence with values placed
    not_replacement = {"~0" : "1", "~1" : "0"}
    for val in not_replacement.keys():
        not_parts = re.search('[\~\(\&\|\-\>]' + val + '[\)\&\|\-\<]', sentence)
        while not_parts != None:
            not_part = not_parts.group(0)
            not_part = not_part[1:-1]
            sentence = sentence.replace(not_part, not_replacement[val])
            sentence = simplify(sentence)
            not_parts = re.search('[\(\&\|\-\>]' + val + '[\)\&\|\-\<]', sentence)
    if ('~0' in sentence) or ('~1' in sentence):
        sentence = replace_negated(sentence)
    return sentence

In [4]:
def simplify(sentence):
    parts = re.search('\([^\~\<\>\(\)\-\&\|]*\)', sentence)
    while parts != None:
        sentence = sentence.replace(parts.group(0), parts.group(0)[1:-1])
        parts = re.search('\([^\~\<\>\(\)\-\&\|]*\)', sentence)
    return sentence

In [5]:
def splitted(sentence, op):
    #check if sentence is of form (P op Q), where P and Q are valid formulas and op is given operation
    if sentence[0] == '(':
        sentence = sentence[1:-1]
    counter = 0
    for idx in range(len(sentence)):
        if sentence[idx] == '(':
            counter += 1
        else:
            if sentence[idx] == ')':
                counter -= 1
            else:
                if (counter == 0) and idx + len(op) <= len(sentence):
                    flag = True
                    for op_idx in range(len(op)):
                        flag = flag and sentence[op_idx + idx] == op[op_idx]
                    if flag:
                        return [construct_sentence(sentence[:idx]), construct_sentence(sentence[idx+len(op):])]
    return None

In [6]:
def construct_sentence(s):
    if s[0] != '(':
        return '(' + s + ')'
    else:
        return s

In [7]:
def evaluate(sentence, model):
    keys = model.keys()
    
    #put variables values
    for key in keys:
        parts = re.search('[~\(\&\|\-\>](' + key + ')[\)\&\|\-\<]', sentence)
        while (parts != None):
            part = parts.group(0)
            part = part[1:-1]
            sentence = sentence.replace(part, model[key])
            parts = re.search('[~\(\&\|\-\>](' + key + ')[\)\&\|\-\<]', sentence)
            
    sentence = simplify(sentence)
    sentence = replace_negated(sentence)
    
    #compute sentence value
    parts = re.search('(\([^\(\)]*\))', sentence)
    while parts != None:
        part = parts.group(0)
        sentence = sentence.replace(part, evaluate_simple(part))
        sentence = replace_negated(sentence)
        sentence = simplify(sentence)
        parts = re.search('(\([^\(\)]*\))', sentence)
    return bool(int(sentence))

In [63]:
def evaluate_clause(clause, model):
    #input: clause x1|x2|x3|...|xn
    clause = construct_sentence(clause)
        
    keys = model.keys()
    
    #put variables values
    for key in keys:
        parts = re.search('[~\(\&\|\-\>](' + key + ')[\)\&\|\-\<]', clause)
        while (parts != None):
            part = parts.group(0)
            part = part[1:-1]
            clause = clause.replace(part, model[key])
            parts = re.search('[~\(\&\|\-\>](' + key + ')[\)\&\|\-\<]', clause)
            
    clause = replace_negated(clause)
    
    if '1' in clause:
        return True
    else:
        return False

In [8]:
def evaluate_simple(sentence):
    if sentence[0] == '(' and sentence[-1] == ')':
        sentence = sentence[1:-1]
    
    #eval and = &
    if sentence == '0&0' or sentence== '0&1' or sentence == '1&0':
        return '0'
    else:
        if sentence == '1&1':
            return '1'
    
    #eval or = |
    if sentence == '1|1' or sentence== '0|1' or sentence == '1|0':
        return '1'
    else:
        if sentence == '0|0':
            return '0'
    
    #eval implication = ->
    if sentence == '0->0' or sentence== '0->1' or sentence == '1->1':
        return '1'
    else:
        if sentence == '1->0':
            return '0'
    
    #eval equivalence = <->
    if sentence == '0<->0' or sentence == '1<->1':
        return '1'
    else:
        if sentence == '1<->0' or sentence == '0<->1':
            return '0'
    
    return sentence

In [9]:
def evaluate_simple_test():
    val = ["0", "1"]
    operators = ['&', '|', '->', '<-', '<->']
    for op in operators:
        for v1 in val:
            for v2 in val:
                sentence = '(' + v1 + op + v2 + ')'
                res = evaluate_simple(sentence)
                print sentence + "   " + res
                sentence = v1 + op + v2
                res = evaluate_simple(sentence)
                print sentence + "   " + res

In [10]:
def extract_symbols(sentence):
    result = []
    symbols = re.search('[\(\&\|\>\-\~][^\(\)\&\|\<\>\-\~]+[\)\&\|\<\-]', sentence)
    while (symbols != None):
        symbol = symbols.group(0)
        symbol = symbol[1:-1]
        result.append(symbol)
        sentence = sentence.replace(symbol, '')
        symbols = re.search('[\(\&\|\>\-\~][^\(\)\&\|\<\>\-\~]+[\)\&\|\<\-]', sentence)
    return result

In [11]:
def tt_entails(KB, sentence):
    #input: KB is list of sentences(str)
    #       sentence is a str
    
    #find all symbols
    symbols = extract_symbols(sentence)
    for s in KB:
        symbols += extract_symbols(s)
    symbols = list(set(symbols))
    
    return tt_check_all(KB, sentence, symbols, {})

In [12]:
def satisfied(KB, model):
    if type(KB) == list:
        return satisfied_kb(KB, model)
    if type(KB) == str:
        return evaluate(KB, model)

def satisfied_kb(KB, model):
    symbols = []
    for s in KB:
        symbols += extract_symbols(s)
    symbols = set(symbols)
    
    if contains(symbols, set(model.keys())):
        result = True
        for s in KB:
            result = result and evaluate(s, model)
        return result
    else:
        return False

In [13]:
def tt_check_all(KB, sentence, symbols, model):
    if len(symbols) == 0:
        if satisfied(KB, model):
            return satisfied(sentence, model)
        else:
            return True
    else:
        result = True
        model[symbols[0]] = '0'
        result = result and tt_check_all(KB, sentence, symbols[1:], model)
        model[symbols[0]] = '1'
        result = result and tt_check_all(KB, sentence, symbols[1:], model)
        return result

In [14]:
string = '(((~(A->B)|C)->D)&F)'

In [18]:
model = {"A" : "0", "B" : "1", "C" : "0", "D" : "0", "F" : "1"}

In [19]:
KB = ['(A|B)', '(B&F)', '(~C->F)']

In [24]:
evaluate(string,model)

True

In [25]:
satisfied(KB,model)

True

In [26]:
KB1 = ['(P)', '(P->Q)']
sentence1 = '(Q)'

In [27]:
tt_entails(KB1, sentence1)

True

In [34]:
def cnf_to_clauses(cnf):
    #given string with cnf conver it to list of clauses
    #note: the resulting clauses have less bracketing then it is required by convert_to_cnf as we allow A|B|C|D|E etc.
    if cnf[0] == '(':
        cnf = cnf[1:-1]
    cnf = cnf.split('&')
    result = []
    for clause in cnf:
        clause = re.sub('\(|\)', '', clause)
        clause = re.sub('(\|+)', '|', clause)
        result.append('(' + clause + ')')
    return result

In [36]:
def convert_to_cnf(sentence):
    #some changes to form of sentence:
    #     replace (v) with v, where v is valiable
    #     add outer brackets if we do not have them
    parts = re.search('\([^\~\<\>\(\)\-\&\|]*\)', sentence)
    while parts != None:
        sentence = sentence.replace(parts.group(0), parts.group(0)[1:-1])
        parts = re.search('\([^\~\<\>\(\)\-\&\|]*\)', sentence)
    if sentence[0] != '(':
        sentence = '(' + sentence + ')'
    sentence = simplify(sentence)
    
    print sentence
    
    #case 1: sentence is just variable:
    parts = re.search('^\([^\~\(\)\<\-\>\&\|]*\)$', sentence)
    if parts != None:
        return sentence
    
    #case 2: sentence is (V1&V2), where Vi are variables
    parts = re.search('^\((\~?[^\(\)\<\>\-\&\|]*)\&(\~?[^\(\)\<\>\-\&\|]*)\)$', sentence)
    if parts != None:
        return '((' + parts.group(1) + ')&(' + parts.group(2) + '))'
    
    #case 3: sentence is (V1&V2), where Vi are formulas
    #parts = re.search('^\((\~?\(.*\))\&(\~?\(.*\))\)$', sentence)
    split = splitted(sentence, '&')
    if split != None:
        return '(' + construct_sentence(convert_to_cnf(split[0])) + '&' + construct_sentence(convert_to_cnf(split[1])) + ')'
    
    #case 4: sentence is (V1|V2), where Vi are variables
    parts = re.search('^\((\~?[^\(\)\<\>\-\&\|]*)\|(\~?[^\(\)\<\>\-\&\|]*)\)$', sentence)
    if parts != None:
        return sentence
    
    #case 5: sentence is (V1|V2), where Vi are formulas
    #parts = re.search('^\((\~?\(.*\))\|(\~?\(.*\))\)$', sentence)
    split = splitted(sentence, '|')
    if split != None:
        #find cnfs for both sides
        cnf_l = construct_sentence(convert_to_cnf(split[0]))
        cnf_r = construct_sentence(convert_to_cnf(split[1]))
        #splits cnf into lists of clauses
        cnf_l = cnf_to_clauses(cnf_l)
        cnf_r = cnf_to_clauses(cnf_r)
        result = '('
        for left_particle in cnf_l:
            for right_particle in cnf_r:
                result = result + '(' + construct_sentence(left_particle) + '|' + construct_sentence(right_particle) + ')&' 
        result = result[:-1]
        result += ')'
        return result
    
    #case 6: sentence is negated: (~V), where V is a varible
    parts = re.search('^\(\~[^\~\<\>\(\)\-\&\|]*\)$', sentence)
    if parts != None:
        return sentence
    
    #case 7: sentence is doubly negated: (~(~V)) or (~(~(V))) or (~~V) or (~~(V)), where V is a formula or variable
    parts = re.search('^\(\~\~(.*)\)$', sentence)
    if parts != None:
        return convert_to_cnf(parts.group(1))
    parts = re.search('^\(\~\(\~(\(.*\))\)\)$', sentence)
    if parts != None:
        return convert_to_cnf(parts.group(1))
    parts = re.search('^\(\~\(\~([^\(\)\|\&\~\<\>\-])\)\)$', sentence)
    if parts != None:
        return convert_to_cnf(parts.group(1))
    
    #case 8: sentence is of form (~(P&Q)), where P and Q are formulas or variables
    parts = re.search('^\(\~(\(.*\&.*\))\)$', sentence)
    if parts != None:
        split = splitted(parts.group(1),'&')
        if split != None:
            return convert_to_cnf('(~' + split[0] + '|~' + split[1] + ')')
    
    #case 9: sentence is of form (~(P|Q)), where P and Q are formulas or variables
    parts = re.search('^\(\~(\(.*\|.*\))\)$', sentence)
    if parts != None:
        split = splitted(parts.group(1),'|')
        if split != None:
            return convert_to_cnf('(~' + split[0] + '&~' + split[1] + ')')
    
    #case 10: sentence is of form (P->Q), where P and Q are formulas or variables
    split = splitted(sentence, '->')
    if split != None:
        return convert_to_cnf('(~' + split[0] + '|' + split[1] + ')')
    
    #case 11: sentence is of form (P<->Q), where P and Q are formulas or variables
    split = splitted(sentence, '<->')
    if split != None:
        return convert_to_cnf('((' + split[0] + '&' + split[1] + ')|(~' + split[0] + '&' + split[1] + '))')
    
    return sentence

In [32]:
def cnf(sentence):
    return cnf_to_clauses(convert_to_cnf(sentence))

In [37]:
cnf(string)

(((~(A->B)|C)->D)&F)
((~(A->B)|C)->D)
(~(~(A->B)|C)|D)
(~(~(A->B)|C))
(~(~(A->B))&~C)
(~(~(A->B)))
(A->B)
(~A|B)
(~C)
D
F


['(~A|B|D)', '(~C|D)', '(F)']

In [70]:
t = re.search('^\((\(.*\))\&(\(.*\))\)$', '((A&B)&(C&D))')

In [71]:
t.group(1)

'(A&B)'

In [187]:
convert_to_cnf('(~(A&B)&(A&B))')

(~(A&B)&(A&B))
(~(A&B))
(~A|~B)
(A&B)


'((~A|~B)&((A)&(B)))'

In [209]:
convert_to_cnf('(~A&~B)')

(~A&~B)


'((~A)&(~B))'

In [218]:
simplify(convert_to_cnf('(((P1&P2)&P3)|Q1)'))

(((P1&P2)&P3)|Q1)
((P1&P2)&P3)
(P1&P2)
P3
Q1


'((P1|Q1)&(P2|Q1)&(P3|Q1))'

In [212]:
convert_to_cnf('((P1&P2)&P3)')

((P1&P2)&P3)
(P1&P2)
P3


'(((P1)&(P2))&(P3))'

In [ ]:
((((P1)|(Q1))&(((P1)|(Q2))&((P2))|(Q1))&((P2))|(Q2))&((P3)|(Q1))&((P3)|(Q2)))

In [197]:
convert_to_cnf('((A->B)&(B|C))')

((A->B)&(B|C))
(A->B)
(~A|B)
(B|C)


'((~A|B)&(B|C))'

In [196]:
convert_to_cnf('(A->B)')

(A->B)
(~A|B)


'(~A|B)'

In [82]:
t = re.search('^\((\~?\(.*\))\&(\~?\(.*\))\)$', '(~(A&B)&(A&B))')

In [89]:
convert_to_cnf(t.group(1))

In [92]:
'((A|B)&(C|D))'[1:-1].split('&')

['(A|B)', '(C|D)']

In [109]:
evaluate('(~~F)', model)

True

In [95]:
model

{'A': '0', 'B': '1', 'C': '0', 'D': '0', 'F': '1'}

In [108]:
replace_negated('(~~1)')

'1'

In [100]:
simplify('(~(0))')

'(~0)'

In [110]:
s = '(~(~((A->B)&(C->D))))'

In [111]:
parts = re.search('^\(\~\(?\~(.*)\)$', s)

In [112]:
parts.group(1)

'((A->B)&(C->D)))'

In [127]:
convert_to_cnf('(~(~A&B))')

(~(~A&B))
(A&B)


'((A)&(B))'

In [118]:
parts = re.search('^\(\~\~(.*)\)$', '(~~A)')

In [119]:
parts.group(1)

'A'

In [120]:
convert_to_cnf('A')

'(A)'

In [141]:
t = re.search('^\((\~?\(.*\))\&(\~?\(.*\))\)$', '(~(A&B)&(A&B))')

In [142]:
t == None

False

In [145]:
t.group(2)

'(A&B)'

In [159]:
t = re.search('^\((\~?\(.*\))\&(\~?\(.*\))\)$', '(~(A&B))')

In [161]:
t

In [74]:
def satisfied_cnf(clauses, model):
    #function that checks if model satisfies list of clauses
    
    result = True
    for c in clauses:
        result = result and evaluate_clause(c,model)
    return result

In [82]:
def walksat(clauses, p=0.5, max_flips=1000):
    #input:
    #       clauses   : list of disjunctive clauses
    #       p         : probability of flip
    #       max_flips : max number of flips allowed
    
    #get symbols
    symbols = []
    for c in clauses:
        symbols += extract_symbols(c)
    symbols = list(set(symbols))
    
    #pick random model
    model = {}
    for s in symbols:
        model[s] = str(random.randint(0,1))
        
    #check if model satisfies set of clauses
    result = satisfied_cnf(clauses, model)
    
    flips = 0
    while (not result) and (flips < max_flips):
        unsatisfied = [c for c in clauses if not evaluate_clause(c,model)]
        
        #pick random unsatisfied clause
        index = random.randint(0,len(unsatisfied) - 1)
        
        c_symbols = extract_symbols(unsatisfied[index])
        
        #pick random symbol
        s_index = random.randint(0, len(c_symbols) - 1)
        
        #flip symbol with probability p
        if random.random() < p:
            if model[c_symbols[s_index]] == '1':
                model[c_symbols[s_index]] = '0'
            else:
                model[c_symbols[s_index]] = '1'
        else:
            unsatisfied_counts = []
            for s in c_symbols:
                if model[s] == '0':
                    model[s] = '1'
                else:
                    model[s] = '0'
                
                unsatisfied_counts.append(len([c for c in clauses if not evaluate_clause(c, model)]))
                
                if model[s] == '0':
                    model[s] = '1'
                else:
                    model[s] = '0'
            s_index = unsatisfied_counts.index(min(unsatisfied_counts))
            s = c_symbols[s_index]
            if model[s] == '0':
                model[s] = '1'
            else:
                model[s] = '0'
                
        flips += 1
    
    return result

In [41]:
replace_negated('(~1|0|0)')

'(0|0|0)'

In [43]:
clauses = ['(~A|B|D)', '(~C|D)', '(F)']

In [64]:
for c in clauses:
    print c, evaluate_clause(c,model)

(~A|B|D) True
['A', 'B', 'D']
(~C|D) True
['C', 'D']
(F) True
['F']


In [53]:
model

{'A': '0', 'B': '1', 'C': '0', 'D': '0', 'F': '1'}

In [71]:
random.randint(0,1)

1

In [75]:
a = {'a':1, 'b':2}

In [76]:
b = a

In [77]:
b['c'] = 3

In [78]:
print a

{'a': 1, 'c': 3, 'b': 2}


In [79]:
[1,2,3,4,5].index(5)

4

In [83]:
walksat(clauses)

True

In [117]:
def examples():
    #1.Modus Ponens
    print "1.Modus Ponens:"
    KB = ['(P)', '(P->Q)']
    alpha = '(Q)'
    if tt_entails(KB, alpha):
        print KB, 'entails ', alpha
    else:
        print KB, 'does not entail ', alpha
        
    #2.Wumpus World
    print "\n\n2.Wumpus World:"
    KB = ['(~P11)', '(B11<->(P12|P21))', '(B21<->((P11|P22)|P31))', '(~B11)', '(B21)']
    alpha = '(P12)'
    if tt_entails(KB, alpha):
        print alpha, " is true"
    else:
        print '~'+alpha, " is true"
        
    #3.Horn Clauses:
    print "\n\n3.Horn Clauses:"
    KB = ['(Mythical->Immortal)', '(~Mythical->~Immortal)', '((Immortal|Mammal)->Horned)', '(Horned->Magical)']
    alpha1 = '(Mythical)'
    alpha2 = '(Magical)'
    alpha3 = '(Horned)'
    if tt_entails(KB, alpha1):
        print "we can prove that it is mythical"
    else:
        print "we can not prove that it is mythical"
    if tt_entails(KB, alpha2):
        print "we can prove that it is magical"
    else:
        print "we can not prove that it is magical"
    if tt_entails(KB, alpha3):
        print "we can prove that it is horned"
    else:
        print "we can not prove that it is horned"
        
    #4.Liars and Truth-tellers (a)
    #Each name variable X here means that "X is truth-teller"
    print "\n\n4.Liars and truth-tellers (a)"
    KB = ['(Amy<->(Cal&Amy))', '(Bob<->~Cal)', '(Cal<->(Bob|~Amy))']
    alpha1 = '(Amy)'
    alpha2 = '(Bob)'
    alpha3 = '(Cal)'
    if tt_entails(KB, alpha1):
        print alpha1, "is truth-teller"
    else:
        print alpha1, "is liar"
    if tt_entails(KB, alpha2):
        print alpha2, "is truth-teller"
    else:
        print alpha2, "is liar"
    if tt_entails(KB, alpha3):
        print alpha3, "is truth-teller"
    else:
        print alpha3, "is liar"
        
    #4.Liars and Truth-tellers (b)
    #Each name variable X here means that "X is truth-teller"
    #Here assuming that "Bob is correct" means "What Bob said is truth", which is equivalent to what Bob said
    print "\n\n4.Liars and truth-tellers (b)"
    KB = ['(Amy<->(~Cal))', '(Bob<->(Amy&Cal))', '(Cal<->(Amy&Cal))']
    alpha = ['(Amy)', '(Bob)', '(Cal)']
    for a in alpha:
        if tt_entails(KB,a):
            print a[1:-1], "is truth-teller"
        if tt_entails(KB, '(~'+a[1:]):
            print a[1:-1], "is liar"
        
    #5.More Liars and Truth-tellers
    #symbols definitions as in previous case
    print "\n\n5.More Liars and truth-tellers"
    KB = ['(Amy<->(Hal&Ida))', 
          '(Bob<->(Amy&Lee))', 
          '(Cal<->(Bob&Gil))', 
          '(Dee<->(Eli&Lee))', 
          '(Eli<->(Cal&Hal))', 
          '(Fay<->(Dee&Ida))', 
          '(Gil<->(~Eli&~Jay))', 
          '(Hal<->(~Fay&~Kay))', 
          '(Ida<->(~Gil&~Kay))', 
          '(Jay<->(~Amy&~Cal))', 
          '(Kay<->(~Dee&~Fay))', 
          '(Lee<->(~Bob&~Jay))']
    alpha = ['(Amy)', '(Bob)', '(Cal)', '(Dee)', '(Eli)', '(Fay)', '(Gil)', '(Hal)', '(Ida)', '(Jay)', '(Kay)', '(Lee)']
    for a in alpha:
        if tt_entails(KB,a):
            print a[1:-1], "is truth-teller"
        if tt_entails(KB, '(~'+a[1:]):
            print a[1:-1], "is liar"
            
    #6.The Doors of Enlightenment (a) 
    #For each x in A, B, C, D, E, F, G, and H we assume x means "x is a knight" and ~x means "x is a knave"
    #For each x in X Y Z we assume x means "x is a good door" and ~x means "x is a bad door"
    #Assume "Either x or y" is exclusive or (despite it is not like that in real life)
    print"\n\n6.The Doors of Enlightenment (a)"
    KB = ['(A<->X)', 
          '(B<->(Y|Z))', 
          '(C<->(A&B))', 
          '(D<->(X&Y))', 
          '(E<->(X&Z))', 
          '(F<->((D&~E)|(~D&E)))',
          '(G<->(C->F))', 
          '(H<->((G&H)->A))']
    alphas = ['(X)', '(Y)', '(Z)']
    for a in alphas:
        if tt_entails(KB,a):
            print "Philosopher can choose door", a[1:-1]
            #"can" here is in case solution is not unique
            
    #6.The Doors of Enlightenment (b) 
    #For each x in A, B, C, D, E, F, G, and H we assume x means "x is a knight" and ~x means "x is a knave"
    #For each x in X Y Z we assume x means "x is a good door" and ~x means "x is a bad door"
    #Assume "Either x or y" is exclusive or (despite it is not like that in real life)
    print"\n\n6.The Doors of Enlightenment (b)"
    KB = ['(A<->X)', 
          '(H<->((G&H)->A))',
          '(C<->(A&((((((B|C)|D)|E)|F)|G)|H)))',
          '(~G->C)']
    #Reasoning for last fact:
    # G: If C is a knight ...
    # If G is a liar then we do not have any information about C as it can take any, depending on ...
    # But if G is a knave, then C->(...) is False which only happens when C=True and (...)=True.
    # As we do not know anything about ..., then G=False means C=True for us.
    alphas = ['(X)', '(Y)', '(Z)']
    for a in alphas:
        if tt_entails(KB,a):
            print "Philosopher can choose door", a[1:-1]
            #"can" here is in case solution is not unique

In [118]:
examples()

1.Modus Ponens:
['(P)', '(P->Q)'] entails  (Q)


2.Wumpus World:
~(P12)  is true


3.Horn Clauses:
we can not prove that it is mythical
we can not prove that it is magical
we can not prove that it is horned


4.Liars and truth-tellers (a)
(Amy) is liar
(Bob) is liar
(Cal) is truth-teller


4.Liars and truth-tellers (b)
Amy is truth-teller
Bob is liar
Cal is liar


5.More Liars and truth-tellers
Amy is liar
Bob is liar
Cal is liar
Dee is liar
Eli is liar
Fay is liar
Gil is liar
Hal is liar
Ida is liar
Jay is truth-teller
Kay is truth-teller
Lee is liar


6.The Doors of Enlightenment (a)
Philosopher can choose door X


6.The Doors of Enlightenment (b)
Philosopher can choose door X


In [114]:
model['E'] = 1
model['G'] = 0
model['H'] = 1
model['X'] = 1
model['Y'] = 0
model['Z'] = 0
print model

{'A': '0', 'C': '0', 'B': '1', 'E': 1, 'D': '0', 'G': 0, 'F': '1', 'H': 1, 'Y': 0, 'X': 1, 'Z': 0}


In [ ]:
evaluate